In [1]:
import sys
sys.path.append('../src')
import os
import glob
import json
import torch
import numpy as np
from semantic_bac_segment.segmentator import Segmentator
from semantic_bac_segment.model_loader import model_loader
import tifffile

# Get images
path_images='../../Semantic_bac_segment/data/source_norm/'
path_masks='../../Semantic_bac_segment/data/masks_cleaned'
image_files = [os.path.join(path_images, file) for file in os.listdir(path_images) if file.endswith('.tiff')]
mask_files = [os.path.join(path_masks, file) for file in os.listdir(path_masks) if file.endswith('.tiff')]

image_files.sort()
mask_files.sort()

max_width, max_height=2400, 2400

for image_path, mask_path in zip(image_files, mask_files):

    print(f'{os.path.basename(image_path)} paired with {os.path.basename(mask_path)}')

coli_mask_frame_1.tiff paired with coli_mask_frame_1.tiff
coli_mask_frame_10.tiff paired with coli_mask_frame_10.tiff
coli_mask_frame_101.tiff paired with coli_mask_frame_101.tiff
coli_mask_frame_109.tiff paired with coli_mask_frame_109.tiff
coli_mask_frame_132.tiff paired with coli_mask_frame_132.tiff
coli_mask_frame_165.tiff paired with coli_mask_frame_165.tiff
coli_mask_frame_166.tiff paired with coli_mask_frame_166.tiff
coli_mask_frame_205.tiff paired with coli_mask_frame_205.tiff
coli_mask_frame_223.tiff paired with coli_mask_frame_223.tiff
coli_mask_frame_232.tiff paired with coli_mask_frame_232.tiff
coli_mask_frame_274.tiff paired with coli_mask_frame_274.tiff
coli_mask_frame_56.tiff paired with coli_mask_frame_56.tiff
mabs_img_01.tiff paired with mabs_img_01.tiff
mabs_img_02.tiff paired with mabs_img_02.tiff
mabs_img_03.tiff paired with mabs_img_03.tiff
mabs_img_04.tiff paired with mabs_img_04.tiff
mabs_img_05.tiff paired with mabs_img_05.tiff
mabs_img_06.tiff paired with mabs_

In [2]:
max_width, max_height=2400, 2400

padded_images = []
padded_masks = []
original_dimensions = {}

for image_path, mask_path in zip(image_files, mask_files):
    print(image_path)
    image_name=os.path.basename(mask_path)
    image = tifffile.imread(image_path)
    mask = tifffile.imread(mask_path)

    height, width = image.shape[:2]
    original_dimensions[image_name] = (height, width)

    pad_height = max_height - height
    pad_width = max_width - width
    
    # Pad the image with zeros
    padded_image = np.pad(image, ((0, pad_height), (0, pad_width)), mode='constant')
    padded_images.append(padded_image)
    
    # Pad the mask with zeros
    padded_mask = np.pad(mask, ((0, pad_height), (0, pad_width)), mode='constant')
    padded_masks.append(padded_mask)

# Convert the padded images and masks to stacks
image_stack = np.stack(padded_images)
mask_stack = np.stack(padded_masks)

# Print the shape of the stacks
print("Image stack shape:", image_stack.shape)
print("Mask stack shape:", mask_stack.shape)

../../Semantic_bac_segment/data/source_norm/coli_mask_frame_1.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_10.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_101.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_109.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_132.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_165.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_166.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_205.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_223.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_232.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_274.tiff
../../Semantic_bac_segment/data/source_norm/coli_mask_frame_56.tiff
../../Semantic_bac_segment/data/source_norm/mabs_img_01.tiff
../../Semantic_bac_segment/data/source_norm/mabs_img_02.tiff
../../Semantic_bac_segment/data/source_norm/mabs_img_0

In [3]:
image_stack=np.expand_dims(image_stack, axis=1)
mask_stack=np.expand_dims(mask_stack, axis=1)

print(f'mask stack shape = {mask_stack.shape}, image stack shape {image_stack.shape}')

mask stack shape = (47, 1, 2400, 2400), image stack shape (47, 1, 2400, 2400)


In [4]:

results_folder='../results'
models_dict = []
for file in os.listdir(results_folder):
    if file.endswith('_model.pth'):
        model_path = os.path.join(results_folder, file)
        config_path = os.path.join(results_folder, file.replace('_model.pth', '_config.json'))

        # Check if the corresponding config file exists
        if os.path.exists(config_path):
            # Add the model and its configuration to the dictionary
            models_dict.append((model_path, config_path))
        else:
            print(f"Warning: Config file not found for model {file}")

models_dict

[('../results/base_unet-multiclass-4_final_model.pth',
  '../results/base_unet-multiclass-4_final_config.json'),
 ('../results/MonaiAttentionUNet-3class-1_final_model.pth',
  '../results/MonaiAttentionUNet-3class-1_final_config.json'),
 ('../results/atomai_unet-3class-3_final_model.pth',
  '../results/atomai_unet-3class-3_final_config.json'),
 ('../results/atomai_unet-3class-2_best_model.pth',
  '../results/atomai_unet-3class-2_best_config.json'),
 ('../results/MonaiUnet-3class-2_final_model.pth',
  '../results/MonaiUnet-3class-2_final_config.json'),
 ('../results/MonaiUnet-multiclass-3_final_model.pth',
  '../results/MonaiUnet-multiclass-3_final_config.json'),
 ('../results/unet_monai_best_model.pth',
  '../results/unet_monai_best_config.json'),
 ('../results/AttentionUNet-2_best_model.pth',
  '../results/AttentionUNet-2_best_config.json'),
 ('../results/MonaiUnet-1_best_model.pth',
  '../results/MonaiUnet-1_best_config.json'),
 ('../results/MonaiUnet-multiclass-4_final_model.pth',
  

In [49]:
import json
from semantic_bac_segment.utils import get_device
for i, params_i in enumerate(models_dict):
  try:
    with open(params_i[1]) as f:
      model_param = json.load(f)
      model_i=model_loader(model_param, get_device(), weights=params_i[0])
  except Exception as e:
    print(f"Error loading model {params_i[0]}: {e}")
    models_dict.pop(i)

In [ ]:
mask_stack_binary=np.where(mask_stack > 20, 1, 0)


In [6]:
from semantic_bac_segment.utils import get_device
from monai.metrics import DiceMetric, MeanIoU
import pandas as pd

monai_dice=DiceMetric(include_background=True, reduction=None)
device=get_device()

for i in models_dict:
    with open(params_i[1]) as f:
        model_param = json.load(f)
        model_i=model_loader(model_param, device)

    segmentator = Segmentator(params_i[0], model_graph=model_i, device=device,  patch_size=256, overlap_ratio=0.25, half_precision=True)
    preds=segmentator.predict(image_stack[:, 0, :,:], sigmoid=True)

    if preds.shape[1]==4:
        preds=preds[:, 1:, :,:]
    
    if preds.shape[1]==3:
        preds=np.max(preds, axis=1)
        preds=np.expand_dims(preds, axis=1)
    
    monai_dice=DiceMetric(include_background=True, reduction=None)
    monai_iou=MeanIoU(include_background=True, reduction=None)

    mask_tensor=torch.from_numpy(mask_stack_binary).float()
    preds_tensor=torch.from_numpy(preds).float()
    
    dice_metrics=monai_dice(mask_tensor, preds_tensor, reduce=None, reduction='none')
    iou_metrics=monai_iou(mask_tensor, preds_tensor, reduce=None, reduction='none')

    iteration_df = pd.DataFrame({
        'model_name': [model_param['model_name']] * len(dice_metrics),
        'position': list(range(len(dice_metrics))),
        'dice_metric': dice_metrics.squeeze().tolist(),
        'iou_metric': iou_metrics.squeeze().tolist()
    })

    metrics_df = pd.concat([metrics_df, iteration_df], ignore_index=True)

